In [ ]:
import os
from ipyfilechooser import FileChooser
import cv2 as cv
import matplotlib.pyplot as plt
import io
import numpy as np
from ipywidgets import interact, fixed, widgets

A function to convert a Numpy array to a binary image that can be used by the ipywidgets.

In [ ]:
def numpy_to_binary(arr):
    is_success, buffer = cv.imencode(".jpg", arr)
    io_buf = io.BytesIO(buffer)
    #print(type(io_buf))
    return io_buf.read()

A file selector to make it easier to load any image from the filesystem. In an application I'd probably accept the path to the file on the command line, use some GUI toolkit to open a file-selector, or in a web application allow the user to upload an image.

In [ ]:
#fc = FileChooser(os.path.dirname(os.getcwd()))
fc = FileChooser("/home/gabor/Dropbox/Data/")
display(fc)

In [ ]:
filename = fc.selected
filename

## Read Image into memory as a numpy array

In [ ]:
original = cv.imread(filename)
type(original)

## Display image using Matplotlib or using the ipywidgets

In [ ]:
#plt.axis("off")
plt.imshow(original) # This image is blue because OpenCV stores the images in BGR order and PyPlot expects RGB
#plt.imshow(cv.cvtColor(original, cv.COLOR_BGR2RGB));
#display(widgets.Image(value=numpy_to_binary(original)));

In [ ]:
original.shape

## Resize the image - Make the image smaller

In [ ]:
def resize_image(img, scale):
    height, width, colors = img.shape
    new_height = int(height * scale)
    new_width = int(width * scale)
    return cv.resize(img, (new_width, new_height), interpolation=cv.INTER_AREA)

In [ ]:
smaller = resize_image(original, 0.5)
display(widgets.Image(value=numpy_to_binary(smaller)));

## Crop image

In [ ]:
smaller.shape

In [ ]:
cropped = smaller[200:600, 0:450] # height, width
display(widgets.Image(value=numpy_to_binary(cropped)));

## Convert to Grayscale

In [ ]:
grey = cv.cvtColor(cropped, cv.COLOR_BGR2GRAY)
display(widgets.Image(value=numpy_to_binary(grey)));

## Blur the image

* [Gaussian blur](https://en.wikipedia.org/wiki/Gaussian_blur) on Wikipedia
* [Blurring or smoothing](https://docs.opencv.org/4.5.3/d4/d13/tutorial_py_filtering.html) OpenCV tutorial

In [ ]:
blurred = cv.GaussianBlur(cropped, ksize=(3, 3), sigmaX=cv.BORDER_DEFAULT)
display(widgets.Image(value=numpy_to_binary(blurred)));

## Edges - Canny

providing two thresholds
* [Edge detection](https://en.wikipedia.org/wiki/Edge_detection)
* [Canny edge detector](https://en.wikipedia.org/wiki/Canny_edge_detector) in wikipedia
* [Canny](https://docs.opencv.org/4.5.3/da/d22/tutorial_py_canny.html) in OpenCV tutorial
* `threshold1` is minimum (required)
* `threshold2` is maximum (required)
* `apertureSize` defaults to 3
* `L2gradient` defaults to `False`

In [ ]:
edges_of_cropped = cv.Canny(cropped, threshold1=125, threshold2=175, apertureSize=3, L2gradient=False)
display(widgets.Image(value=numpy_to_binary(edges_of_cropped)));

In [ ]:
edges_of_blurred = cv.Canny(blurred, threshold1=125, threshold2=175, apertureSize=3, L2gradient=False)
display(widgets.Image(value=numpy_to_binary(edges_of_blurred)));

## Dilation (Morphology)


* dilate means to expand, in this context it is the opposite of erosion
* adds pixels to the boundaries of an image

* [Dilation](https://en.wikipedia.org/wiki/Dilation_(morphology)) in Wikipedia
* [Dilation and erosion](https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html) in OpenCV tutorial


In [ ]:
dilated = cv.dilate(edges_of_blurred, (7,7), iterations=3)
display(widgets.Image(value=numpy_to_binary(dilated)));

## Affine Transformations

* Rotations (linear transformation)
* Translations (vector addition)
* Scale operations (linear transformation)


* [Wrap Affine](https://docs.opencv.org/4.5.2/d4/d61/tutorial_warp_affine.html) OpenCV tutorial
* [Affine Transformations](https://en.wikipedia.org/wiki/Affine_transformation) in Wikipedia

# Translation - shifting image

In [ ]:
def translate(img, x, y):
    translation_matrix = np.float32([[1, 0, x], [0, 1, y]])
    dimensions = (img.shape[1], img.shape[0])
    return cv.warpAffine(img, translation_matrix, dimensions)

translated = translate(cropped, 100, -100)
display(widgets.Image(value=numpy_to_binary(translated)));